# COMP41680 Data Science in Python Assignment 1

<b>Name:DARSHAN HARESH VIRA</b>

<b>Student ID: 17204812</b>

In [0]:
import pandas as pd
import numpy as np


## Task 1: Identify one or more suitable web APIs
<p><b><i><font color="red">API Chosen:</font></i> </b></p>
1. A single API that was chosen for this assignment was that provided by <strong>http://www.omdbapi.com</strong></p>
2. The API is not available for free but they give a free account upon signing up, this entitles the user to 1000 calls to the API per day.
3. The API key I received which works here is *APIKEY* , *APIKEY*. Both work perfectly fine. So if while checking one API key is not working or is exhausted you can use the second as well.


In [0]:
apiKey = "APIKEY"
#apiKey2="APIKEY"


## Task 2: Collect data your chosen API
<font color="red"><b><i>Collecting Raw Data - directly into DataFrame:</i> read_json() of Pandas Library!</font></b></p>
The following function is written to get each movie data through one api call each. So by recursive calling of the fucntion, the entire dataframe having movie data is constructed. The function is commented throughout and is called below in Task3:

In [0]:
#api-apikey
#api2- apikey
import pandas as pd
#variable data will hold all the movie data obtained by all the apicall
data = pd.DataFrame()
def apicall(movieId):
    global data #global tells the fucntion to access the global variable!
    try:
        movieId=movieId[2:]
        #the movie id obtained from movies.csv file(have defined what is this file about later)
        #has "tt" at the start of each id, so taking only substring of the id and ignoring tt
        if len(movieId) == 7: #just a final check that id length is equal to 7 as API needs a key which is 7 integers long
            url = "http://www.omdbapi.com/?i=tt" + movieId + "&apikey="+apiKey
        else:
            print("Invalid Key")
        #reading the json data from url directly to pandas dataframe
        data=data.append(pd.read_json(url))
    except Exception as e:
        print(e)

## Task 3: Parse the collected data, and get data stored in an appropriate file format
<p><b><i><font color="red">Calling the function and Writing collected data to CSV:</font></i> </b></p>
The following code retreives the data using the above function apicall and writes it to a DataFrame and then exports the data to a csv file. This data needs extensive cleaning and manipulation before it can be used.</p>


### There are two parts:

<strong> <i><u>Part 1: Reading the MovieIds from movies.csv file</u></i></strong>

I have downloaded a dataset(CSV File) having IMDB movie ids, which are needed to make api call to omdb,from a github repository. I used movie id column from one csv file and took movie title from another to create a new CSV file having movie titles and IMDB ids which is read by the pyhton program below.
 
 <u>Note</u>: The csv file, "movies.csv" is self created by taking 2 CSVs files downloaded from github repositiories. So no code has been included below. The code directly uses the movies.csv file.

<strong><i><u> Part 2: Making the API call using apicall fucntion for each movie</u></i></strong>

Function apicall is called to make an api call for each movie obtained from the csv file. Then the json object is parsed directly into a DataFrame and then later, the entire dataframe is written to csv file!
<p><u>Note</u>: The imdb movie id obtained from csv file. I have added the code to check the movie id length and then make a call to api!</p>

The api is called in the code below and the collected data is stored in a csv file:

In [0]:
#csv reading and api calling!
df=pd.read_csv("movies.csv")
a=df['imdb_id']#getting the imdb_id column as series
for i in a[:277]:
    apicall(str(i))#calling the api for each movie's IMDBb Id read from movies.csv
data=data.set_index("Title") #setting the index of DF
data.to_csv("CollectedData.csv")
#it takes a bit long at times to execute the query and might even show HTTP Error 524: Origin Time-out
#but the data gets loaded

## Task 4: Load and Apply any pre-processing steps to clean/filter/combine the data

First we need to see the data collected

In [0]:
data

<p><b><i><font color="red">Duplicate Values</font></i> </b></p>

We can see some data is repeated twice and some even thrice. On careful examining the data obtained, we see the column ratings causes this duplication. Ratings have:
a) source as IMDB
b) source as RottenTomatoes
c) sources as Metacritic

Some movies have just one of three, some have two, some have all three.

In [0]:
data['Ratings']

As we have a column named 'imdbRating' which has IMDB movie ratings(same data as 'Source:IMDB' of 'Ratings') and would be sufficient enough for the current analysis and thus, Ratings column can be deleted.

In [0]:
del data['Ratings']

But this doesn't solve the problem entirely. It just removes the column creating duplicates. To remove duplicates, I have used drop_duplicates method of Pandas and criteria to indentify duplicates can be imdbIDs of each movies.

In [0]:
data=data.drop_duplicates(subset="imdbID") #subset tells which column should be used to identify two duplicates

In [0]:
data

<p><b><i><font color="red">Data Reduction- Removing Redundant Columns</font></i> </b></p>

Before plunging into further cleaning of the data we can get rid of the columns which wont be of any use to the current data analysis. The columns like,Plot,Poster,Response,Website,imdbID can be straight away deleted from the dataframe.

In [0]:
data.drop(['Plot','Poster','Response','Website','imdbID'],axis=1,inplace=True)

<p><b><i><font color="red">Missing Value Columns</font></i> </b></p>

In [0]:
pd.isnull(data).sum()

Though we see N/A values in the columns, the result of pd.isnull(data).sum() is 0 for almost all the columns, which means the data we have as N/A is in String format and not sepcifically null or N/A. To be able to count such values in each column, I have used replace function of Pandas and np.nan of Numpy

In [0]:
data.replace('N/A', np.nan,inplace=True)

In [0]:
pd.isnull(data).sum()

Starting with columns like <strong>Actors, Country, Director, Genre, Language, Released, Runtime, Writer</strong> have only single-digit or a small double digit value, N/A rows out of 200 and must be replaced with general term such as 'Not Available' or 'Unknown'. 

In [0]:
data[['Actors', 'Country', 'Director', 'Genre', 'Language', 'Released', 'Runtime', 'Writer']]=data[['Actors', 'Country', 'Director', 'Genre', 'Language', 'Released', 'Runtime', 'Writer']].fillna(value="Unknown")

Column <strong>Awards</strong> has N/A values which even represents 0 awards and 0 nominations for the film and hence the N/A values should be replaced with 0 instead of N/A

In [0]:
data['Awards'].replace(np.nan,'0',inplace=True)

By the rule of the thumb for columns having more than 60% data as null, they should be dropped from the table. In our case, the columns like <strong>BoxOffice, DVD, Metascore,totalSeasons</strong> must be dropped as it might produce skewed analysis if the missing values are replaced by some global constant or any central tendency as a huge amount of data will have same values.

In [0]:
data.drop(['BoxOffice','DVD','Metascore','totalSeasons'],axis=1,inplace=True)

The column <strong>Production</strong> has name of the production house that made the film. This column does have 52%(48% missing) of the records with values and hence can't be ignored. The value is not numerical hence cant be replaced with central tendencies as well. So the last resort is replacing null values with a global constant again. 

In [0]:
data['Production']=data['Production'].fillna(value="Unknown")

Coming to the only remaining column with null values i.e. Ratings. Let us first find out the distinct values available in ratings.

In [0]:
ratings_counts = data["Rated"].value_counts()
ratings_counts

First we need to bring some consistency in the existing distinct values. We can see that the column has two values,<strong> Unrated</strong> and even <strong>Not Rated</strong>. It is better to keep any one of the two as on researching further about it I found, "Not Rated and Unrated is just different ways of saying the MPAA never assigned a rating to that particular cut".

In [0]:
data['Rated'].replace('UNRATED','NOT RATED',inplace=True)

Given that the movie which is not rated has 'Not Rated' value, all the N/A values in the column Rated can be replaced with 'Not Rated' value.

In [0]:
data['Rated'].replace(np.nan,'NOT RATED',inplace=True)

So after treating the null values of the dataset, the resulting dataframe will not have any null values. Lets check it again!

In [0]:
pd.isnull(data).sum()

<p><b><i><font color="red">Feature Transformation</font></i> </b></p>

For graph plotting and further analysis, we need the numeric column to be in numeric format and not in string format.

In [0]:
#Conversion of Numeric Values in String format to Numberic format
data['imdbRating']=pd.to_numeric(data.imdbRating)
#String to a List
data['Awards']=data['Awards'].str.replace(' & ',',') #Awarsd format is 'x wins & y nominated' or 'y nominated' 
#so first remove ' & ' 
data['Awards']=data['Awards'].str.replace(' ',',')#then the sapce between 'x wins' and between 'y nominated'
data['Awards']=data['Awards'].str.split(',')#then into List. Reason for this conversion will see in Attribute Construction

Even columns <b>imdbVotes</b> and <b>Year</b> both have numeric values. But there's a twist to it.

<i><u>Problem with imdbVotes:</u></i>
The column imdb votes has values in sigle,double,triple and even quadraple digits values. as the value goes beyong 999, there's a comma in values and those values are read as string and rest of the values as int. So there's a mixture of values in the column.(found out by type() method on each value of the column)

<i><u>Problem with Year:</u></i>
Only one cell of the year column has value as '2006-' for some reason

In [0]:
data['imdbVotes']=data.imdbVotes.astype(str) #Converting Entire column to String
data['imdbVotes']=data['imdbVotes'].str.replace(",","") #Removing the commas
data['imdbVotes']=pd.to_numeric(data.imdbVotes) #Converting back to Numeric Value!

In [0]:
data['Year'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'') #keeps only digits, so "-" 
#or anything for that matter, expect numbers, gets deleted
data['Year']=pd.to_numeric(data.Year) #Because of the string in '2006-' we need to apply to_numeric to entire column to the entire column

Even the column Runtime can be converted into numeric value column but first we need to remove mins from each of the value of that column and as 'mins' is common for all the values, we can rename the column as 'Runtime_mins' and then convert.

In [0]:
data['Runtime'].replace(regex=True,inplace=True,to_replace=r'\D',value=r'') #keeps only digits nothing expect digits,
#so mins gets deleted
data['Runtime']=pd.to_numeric(data.Runtime) #type casting Column to int
data.rename(index=str, columns={"Runtime": "Runtime_mins"},inplace=True) #Renaming column
data['Runtime_mins'].fillna(0,inplace=True) #as 'unknown' values get replaced by null by the regex function

In [0]:
pd.isnull(data).sum()

<p><b><i><font color="red">Feature Construction</font></i> </b></p>

Attribute construction refers to replacing or adding new attributes inferred by existing attributes. In our case this feature can be used on Awards column

In [0]:
data['Wins'] = 0 #creating two columns Wins and Nominated with 0 as deafult values for all rows
data['Nominated'] = 0
for index, row in data.iterrows(): #itterating over the rows of data DataFrame
    if (len(row['Awards']) == 1): #if Awards has 0 as the value,keep Wins=0,Nominated=0
        continue
    elif(len(row.Awards)==2):#if Awards has 'x nominated' OR 'x win' as the value,update Nominated=x OR Wins=x respectively
        l=row.Awards[1]
        if l[0]=='w':#if the value is x win
            data.loc[index, 'Wins'] = row.Awards[0]
        else:#if the value is x Nominated
            data.loc[index, 'Nominated']= row.Awards[0]
    else:#if Awards has x wins,y nominated as the value,update Wins=x and Nominated=y
        l=row.Awards
        data.loc[index,'Wins']=l[0]
        data.loc[index,'Nominated']=l[2]
data['Wins']=data.Wins.astype(int) #Coverting the column type as int
data['Nominated']=data.Nominated.astype(int) #Coverting the column type as int
del data['Awards'] #drop the Awards column

<p><b><i><font color="red">Sorting the clean data</font></i> </b></p>

In [0]:
data=data.sort_values(by=['Year'], ascending=[True]) #Sorting the data by Year

In [0]:
data.to_csv("CleanedData.csv")

## Task 5:  Analyse and summarise the cleaned dataset

<p><b><i><font color="red">Matplotlib and Pandas Graphing</font></i> </b></p>

In [0]:
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

Before going into specific analysis, lets see the basic one

In [0]:
data.describe()

As the countries and genre are present as String(few movies have multiple countries,multiple genres), We need to check the count for individual countries as follows:
   

In [0]:
#Before proceeding further with the analysis, lets see which countries have the most number of entries in the dataset
print("Country counts in the dataset")
print("UK: ",data['Country'].str.contains('UK').sum())
print("USA: ",data['Country'].str.contains('USA').sum())
print("India: ",data['Country'].str.contains('India').sum())
print("France: ",data['Country'].str.contains('France').sum())
print("Germany: ",data['Country'].str.contains('Germany').sum())
print("Ireland: ",data['Country'].str.contains('Ireland').sum())
print("Russia: ",data['Country'].str.contains('Russia').sum())
print("Australia: ",data['Country'].str.contains('Australia').sum())
print("China: ",data['Country'].str.contains('China').sum())
print("Japan: ",data['Country'].str.contains('Japan').sum())
print("Canada: ",data['Country'].str.contains('Canada').sum())
print("Austria: ",data['Country'].str.contains('Austria').sum())
print("Italy: ",data['Country'].str.contains('Italy').sum())

In [0]:
#Before proceeding further with the analysis, lets see which countries have the most number of entries in the dataset
print("Genre counts in the dataset")
print("Drama: ",data['Genre'].str.contains('Drama').sum())
print("Comedy: ",data['Genre'].str.contains('Comedy').sum())
print("Action: ",data['Genre'].str.contains('Action').sum())
print("Sci-Fi: ",data['Genre'].str.contains('Sci-Fi').sum())
print("Crime: ",data['Genre'].str.contains('Crime').sum())
print("Thriller: ",data['Genre'].str.contains('Thriller').sum())
print("Documentary: ",data['Genre'].str.contains('Documentary').sum())
print("Short: ",data['Genre'].str.contains('Short').sum())
print("Horror: ",data['Genre'].str.contains('Horror').sum())
print("Mystery: ",data['Genre'].str.contains('Mystery').sum())
print("War: ",data['Genre'].str.contains('War').sum())
print("Musical: ",data['Genre'].str.contains('Musical').sum())
print("Advnture: ",data['Genre'].str.contains('Adventure').sum())
print("Family: ",data['Genre'].str.contains('Family').sum())

As we can see there are highest number of entries are for USA and then India and genre as drama,comedy and action, we will use these values in majority part of our further analysis.

<p><b><i><font color="red">Genre Ratings over the years in USA and India</font></i> </b></p>

In [0]:
plt.figure(figsize=(16,6))
#creating a DF havind Drama movies info and country as USA
US_drama=data[data.Country.str.contains('USA') & data.Genre.str.contains('Drama') &(data.Year>1990)]
#Plotting the Year, imdbRatings to get Ratings of the movies over the years
plt.plot(US_drama.Year,US_drama.imdbRating,label="Drama")
#creating a DF havind Comedy movies info and country as USA
US_comedy=data[data.Country.str.contains('USA') & data.Genre.str.contains('Comedy') &(data.Year>1990)]
plt.plot(US_comedy.Year,US_comedy.imdbRating,label="Comedy")
#creating a DF havind Action movies info and country as USA
US_action=data[data.Country.str.contains('USA') & data.Genre.str.contains('Action') &(data.Year>1990)]
plt.plot(US_action.Year,US_action.imdbRating,label="Action")
plt.xlabel('Years')#X axis label
plt.ylabel('Ratings')#Y axis label
#mentioning where the legend will appear and style
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=1, borderaxespad=0.)
plt.title("IMDb Ratings of Drama, Comedy and Action in US over the Years") #title of the plot
plt.show()
print("Max rating of Drama Genre:",US_drama.imdbRating.max())
print("Max rating of Comedy Genre:",US_comedy.imdbRating.max())
print("Max rating of Action Genre:",US_action.imdbRating.max())

1. As seen from the graph in the year 1995 in USA, the genres, Drama Action,Comedy had same positive rating, but Action was less popular in comparison.
2. Towards the year 1998-1999, Action movies had better ratings but tables turned for the periods from 2000 to 2005. During this period though, Drama movies had more rating than comedy, its rating was on decline and conitnued to decline till 2010. 
3. But around 2011 and 2012, drama movies had major spikes in ratings. By the end of the graph period, Drama and Comedy movie ratings are showing increasing trend , Action movie ratings have decreasing ratings in comparison.

<B> Comedy stays consistently positive rated genre in US and Comedy and Drama genres are more preferred than Action genre.</B>

In [0]:
plt.figure(figsize=(18,6)) #plot figure size
#creating a DF havind Drama movies info and country as India
India_drama=data[data.Country.str.contains('India') & data.Genre.str.contains('Drama')]
#Plotting the Year, imdbRatings to get Ratings of the movies over the years
plt.plot(India_drama.Year,India_drama.imdbRating,label="Drama",)
#creating a DF havind Action movies info and country as India
India_comedy=data[data.Country.str.contains('India') & data.Genre.str.contains('Comedy') &(data.Year>1990)]
plt.plot(India_comedy.Year,India_comedy.imdbRating,label="Comedy")
#creating a DF havind Action movies info and country as India
India_action=data[data.Country.str.contains('India') & data.Genre.str.contains('Action') &(data.Year>1990)]
plt.plot(India_action.Year,India_action.imdbRating,label="Action")
plt.xlabel('Years') #X axis label
plt.ylabel('Ratings')#Y axis label
#mentioning where the legend will appear and style
plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3,
           ncol=1, borderaxespad=0.)
plt.title("IMDb Ratings of Drama, Comedy and Action in India over the Years") #title of the plot
plt.show()
print("Max rating of Drama Genre:",India_drama.imdbRating.max())
print("Max rating of Comedy Genre:",India_comedy.imdbRating.max())
print("Max rating of Action Genre:",India_action.imdbRating.max())

1. The dataset contains Drama ratings for the Indian cinema since 1970. Rest of the two categories have movie ratings from around 1990.
2. Drama movie ratings did have a few major spikes in the year range of 1990-2000 but has minor increase and decrease from 2000-2010 and  a sharp increase near the end of the graph.
3. Comedy movie ratings has a gradual increase over the years with minor decreases. Increases rapidly after year 2010 till the end.
4. Action movie ratings has sharp ups and downs and not very consistent till 2000. But after 2000, Action movie ratings have increased steadily till 2010 and decreases after the year 2010.
 
<b> Considering the fact that no genre ratings reached as high as Drama ratings in India, it is safe to say that Drama is a more preferred genre in India compared to Action and Comedy.</b>

<p><b><i><font color="red">Runtime over the years in USA and India</font></i> </b></p>

In [0]:
plt.figure(figsize=(16,6)) #plot figure size
US_movies=data[data.Country.str.contains('USA') & data.Runtime_mins > 0] #as the N/A values were replaced with 0
plt.plot(US_movies.Year,US_movies.Runtime_mins,label="Action")
plt.xlabel('Years') #X axis label
plt.ylabel('Runtime')#Y axis label
plt.title("Runtime of movies in USA over the Years") #title of the plot
plt.show()
print("The mean of the runtime in USA for the three genres are:")
print("Drama:",US_drama['Runtime_mins'].mean())
print("Comedy:",US_comedy['Runtime_mins'].mean())
print("Action:",US_action['Runtime_mins'].mean())
print("Range of the Runtime:",US_movies.Runtime_mins.min()," to ",US_movies.Runtime_mins.max())
print("Average of the runtime of all the US movies:",US_movies['Runtime_mins'].mean())

In the start the US movies were a bit shorter than they are in recent years. But the US movie runtime has increased just a bit more over the years compared to the start, but not by much. US movies have average length of 1.5-2 hours.

In [0]:
plt.figure(figsize=(16,6)) #plot figure size
India_movies=data[data.Country.str.contains('India') & data.Runtime_mins>0]
plt.plot(India_movies.Year,India_movies.Runtime_mins)
#plotting runtime changes over the years
plt.xlabel('Years') #X axis label
plt.ylabel('Runtime')#Y axis label
plt.title("Runtime of movies in India over the Years") #title of the plot
plt.show()
print("The mean of the runtime in India for the three genres are:")
print("Drama:",India_drama['Runtime_mins'].mean())
print("Comedy:",India_comedy['Runtime_mins'].mean())
print("Action:",India_action['Runtime_mins'].mean())
print("Range of the Runtime of Indian Movies:",India_movies.Runtime_mins.min()," to ",India_movies.Runtime_mins.max())
print("Average of the runtime of all the Indian movies:",India_movies['Runtime_mins'].mean())

As we can see from the graph, the Indian movies' Runtime has decreased over the years. In early days, the movies in India were about 180-220 mins long. But in recent times the runtime of Indian movies has reduced to be in between 2.5-3hrs and is decreasing gradually over the years.

<p><b><i><font color="red">Award Won over the years by USA and Indian movies</font></i> </b></p>

In [0]:
plt.figure(figsize=(12,8))
plt.subplot(2, 1, 1)
US_awards=US_movies.groupby(['Year']).sum() #first we need to group data based on year
award_plot1=US_awards[US_awards['Wins']>0]['Wins'].plot.bar() 
award_plot1.set_title("Awards Won by US movies over the years")
award_plot1.set_ylabel("Wins")
plt.show()
plt.figure(figsize=(12,12))
plt.subplot(3, 1, 2)
India_awards=India_movies.groupby(['Year']).sum() #first we need to group data based on years
award_plot2=India_awards[India_awards['Wins']>0]['Wins'].plot.bar() 
award_plot2.set_title("Awards Won by Indian movies over the years")
award_plot2.set_ylabel("Wins")
plt.show()
print("Total awards won by US movies:",US_movies.Wins.sum())
print("Total awards won by Indian movies:",India_movies.Wins.sum())
#Number of Wins per Year, if the 0 wins for the year then ignore that

1. As seen from the graph, the year 2015 is the most successful year for the US movies. The movies in our dataset show this trend, where the wins of US movies is increasing from year 2013 and has highest win of 35 awards in 2015.
2. For the Indian movies, the data shows that year 2000 was the most successful year as Indian movies got more than 20 awards that year. 
3. Even though US movies are greater in number in comparison to Indian movies in the dataset, Indian movies have bagged a greater amount of awards in comparison. 

<p><b><i><font color="red">Relation between Attributes</font></i> </b></p>

To check the relation between different attributes, we need to first visualise it using a scatter matrix.

In [0]:
from pandas.plotting import scatter_matrix
scatter_matrix(data, alpha=0.7, figsize=(8, 8),diagonal='kde')

From the above Scatter Matrix on careful analysis, we can see that there exist some relation between IMDb Ratings and Wins & IMDb Ratings and Nominations.

<p><b><i><font color="red">Relation between IMDb Ratings and Awards won,IMDb Ratings and Nominations earned by the movies</font></i> </b></p>

In [0]:
plt.figure(figsize=(10,10))
plt.subplot(2, 1, 1)
plt.scatter(data['imdbRating'],data['Wins'])
plt.xlabel('IMDb Ratings')
plt.ylabel('Number of Awards Won')
plt.title('Relationship between Ratings and Awards')
plt.figure(figsize=(10,10))
plt.subplot(2, 1, 2)
plt.scatter(data['imdbRating'],data['Nominated'])
plt.xlabel('IMDb Ratings')
plt.ylabel('Number of Nominations')
plt.title('Relationship between Ratings and Nominations')

<b>Awards</b>
* It can be seen there's a rising exponential curve but its not always true.
* Some movies will have higher ratings and higher awards but some might have lower ratings and still large number of awards and vice versa. 
* This is because the awards are given by critics but IMDb Ratings are given by the users/public. So there might be difference of opinion between the two and our dataset highlights this difference!

<b>Nomination</b>
* It can be seen the shape of the scatter plot follows a normal distribution with a bell shaped curve.

<b><i><font color=red>Relation between Rated and imdbRating </font></i></b>

In [0]:
plt.figure(figsize=(10,10))
plt.subplot(2, 1, 1)
a=data[~(data['Rated'].str.contains('NOT RATED'))] # all entries except Not Rated
plt.barh(a.Rated,a.imdbRating)
plt.xlabel('IMDb Ratings')
plt.ylabel('Movie Rated as')
plt.title('Relationship between Movie Rating/Certificate and IMDb Ratings')

<b>Movies with PG ratings have the highest ratings followed by R rated and TV-14 movies, respectively<b>

<b><font color=red> Summary </font></b>

* The dataset had majority of US and Indian based movies with top genres as Drama,Comedy,Action.
* Comedy and Drama genres are more preferred in US than Action genre.
* Drama is preferred more in India than Comedy and Action.
* Wins and IMDb Ratings have approximately rising exponential curve and Nominations and IMDb have a bell shaped normal curve.
* Runtime has decreased over the years for India severly. India made longer movies compared to US. US have made movies of almost same length since a long period of time(slight increase or decrease periodically).
* PG rated movies have highest IMDb Ratings.
* Further analysis between <b> Revenue and Awards Wins and Nominations, Revenue and IMDb Ratings, Box office collection based on countries, Highest Box Collection based on Language, Country, Production House</b> could be made if BoxOffice, Budget information of all the movie is available